In [3]:
from weaPoints import WeaPoints, InterpolationMethod
from anemosData import TsNcWindData, WindDataKind
import numpy as np
import xarray
import pandas

# Loading power curve

In [2]:
path = r"./wea_data/example/test_wea.nc"
power_curves = xarray.open_dataset(path)
power_curves["test_wea"]

<xarray.DataArray 'test_wea' (wspd: 24502, rho: 3251)>
[79656002 values with dtype=float64]
Coordinates:
  * wspd     (wspd) float64 0.0 0.001 0.002 0.003 0.004 ... 24.5 24.5 24.5 24.5
  * rho      (rho) float64 0.95 0.9501 0.9502 0.9503 ... 1.275 1.275 1.275 1.275
Attributes:
    describtion:    interpolated Leistungskennlinie
    creation_date:  2022-01-25
    author:         Claudius Wehner

# Loading Points

In [3]:
size = 10
lon = np.around(np.random.uniform(6.0, 14.0, size=size),5)
lat = np.around(np.random.uniform(48.0, 55.0, size=size),5)
lat_lon_coor = np.stack((lat, lon), axis=1)
level = np.around(np.random.uniform(80.0, 140.0, size=size),1)

testPoints = WeaPoints(
    lat_lon_coor = lat_lon_coor,
    level = level,
    wea_types = ["test_wea"] * size,
    #interpolation_method = [InterpolationMethod.LINEAR] * size,
)

testPoints.point_list[:5]

[_WeaPoint(lat_lon_coor=array([53.95061,  8.11155]), level=132.4, wea_type='test_wea', x_y_coor=[-171663.07550086264, 342670.84680363646]),
 _WeaPoint(lat_lon_coor=array([51.03681, 11.99585]), level=121.4, wea_type='test_wea', x_y_coor=[87951.9832082702, 16718.36316498868]),
 _WeaPoint(lat_lon_coor=array([52.3646 , 12.19515]), level=117.2, wea_type='test_wea', x_y_coor=[98944.67426487707, 164377.5011899491]),
 _WeaPoint(lat_lon_coor=array([52.17118,  7.92576]), level=113.6, wea_type='test_wea', x_y_coor=[-191347.00595121377, 145567.31260115516]),
 _WeaPoint(lat_lon_coor=array([48.02506,  7.26377]), level=136.6, wea_type='test_wea', x_y_coor=[-258080.20913287203, -312482.9069739764])]

# Get Time series

In [4]:
testPoints.get_power_output(power_curves=power_curves, time_frame=[2009,"2014-09-20"])

Passed time_frame valid!
time loaded
Windpower turbine 1 complete
Windpower turbine 2 complete
Windpower turbine 3 complete
Windpower turbine 4 complete
Windpower turbine 5 complete
Windpower turbine 6 complete
Windpower turbine 7 complete
Windpower turbine 8 complete
Windpower turbine 9 complete
Windpower turbine 10 complete


In [1]:
Eout = {}
for num, point in enumerate(testPoints.point_list):
    Eout[f"wea_{num+1}: {point.lat_lon_coor}"] = point.power_time_series

df = pandas.DataFrame(data=Eout,index=testPoints.time)
df.head()#.resample("14d").mean().plot(figsize=(22,6))

NameError: name 'testPoints' is not defined

# 90m mean

In [102]:
import numpy as np

def weibull (v_i:np.array, A:float, k:float) -> np.array:
        return 1 - np.exp(-(v_i/A)**k)


def weibull_power(lkl, A, k, rho, s:float=0.85) -> float:
        F = weibull(v_i=lkl.wspd.to_numpy(), A=A, k=k)
        P = lkl.interp(rho=rho, method="linear").to_numpy()

        AEP_list = list()
        for i, _ in enumerate(F):
                if i == 0:
                        AEP_i = 8760 * F[i] * P[i]

                else:
                        AEP_i = 8760 * (F[i] - F[i-1]) * ((P[i] + P[i-1])/2)
                
                AEP_list.append(AEP_i)

        return sum(AEP_list) * s

In [103]:
path = r"/home/eouser/Documents/code/Windatlas/windatlas/anemos_data/wea_data/example/WEA_beispiel.csv"
power_curves = pandas.read_csv(path, delimiter=";", index_col=0)
power_curves = power_curves.reindex(sorted(power_curves.columns), axis=1)

rho = 1.227
k = 2
A = 6
v_mean = 7.7

lkl = xarray.DataArray(power_curves).rename({"dim_0":"wspd", "dim_1":"rho"})
lkl = lkl.assign_coords(rho=np.float64(lkl.rho))

print(weibull_power(lkl, A, k, rho, s=1))

10158411.857001599
